# LangGraph 101

[LLMs](https://python.langchain.com/docs/concepts/chat_models/) tornam possível incorporar inteligência em uma nova classe de aplicações. [LangGraph](https://langchain-ai.github.io/langgraph/) é um framework para ajudar a construir aplicações com LLMs. Aqui, vamos revisar o básico do LangGraph, explicar seus benefícios, mostrar como usá-lo para construir fluxos de trabalho / agentes, e mostrar como funciona com [LangChain](https://www.langchain.com/) / [LangSmith](https://docs.smith.langchain.com/).

![ecosystem](./img/ecosystem.png)

## Modelos de chat

[Modelos de chat](https://python.langchain.com/docs/concepts/chat_models/) são a base das aplicações LLM. Eles são tipicamente acessados através de uma interface de chat que recebe uma lista de [mensagens](https://python.langchain.com/docs/concepts/messages/) como entrada e retorna uma [mensagem](https://python.langchain.com/docs/concepts/messages/) como saída. LangChain fornece [uma interface padronizada para modelos de chat](https://python.langchain.com/api_reference/langchain/chat_models/langchain.chat_models.base.init_chat_model.html), facilitando [acessar muitos provedores diferentes](https://python.langchain.com/docs/integrations/chat/).

In [ ]:
from dotenv import load_dotenv
load_dotenv("../.env", override=True)

In [ ]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("gemini-2.5-flash", model_provider="google-genai", temperature=0)

## Executando o modelo

A interface `init_chat_model` fornece métodos [padronizados](https://python.langchain.com/docs/concepts/runnables/) para usar modelos de chat, que incluem:
- `invoke()`: Uma única entrada é transformada em uma saída.
- `stream()`: As saídas são [transmitidas](https://python.langchain.com/docs/concepts/streaming/#stream-and-astream) conforme são produzidas.

In [ ]:
result = llm.invoke("O que é um agente?")

In [ ]:
type(result)

In [ ]:
from rich.markdown import Markdown
Markdown(result.content)

## Ferramentas

[Ferramentas](https://python.langchain.com/docs/concepts/tools/) são utilitários que podem ser chamados por um modelo de chat. No LangChain, criar ferramentas pode ser feito usando o decorador `@tool`, que transforma funções Python em ferramentas chamáveis. Ele automaticamente inferirá o nome da ferramenta, descrição e argumentos esperados da definição da função. Você também pode usar [servidores Model Context Protocol (MCP)](https://github.com/langchain-ai/langchain-mcp-adapters) como ferramentas compatíveis com LangChain.

In [ ]:
from langchain.tools import tool

@tool
def write_email(to: str, subject: str, content: str) -> str:
    """Escrever e enviar um email."""
    # Resposta placeholder - em aplicação real enviaria email
    return f"Email enviado para {to} com assunto '{subject}' e conteúdo: {content}"

In [ ]:
type(write_email)

In [ ]:
write_email.args

In [ ]:
Markdown(write_email.description)

## Chamada de Ferramentas

Ferramentas podem ser [chamadas](https://python.langchain.com/docs/concepts/tool_calling/) por LLMs. Quando uma ferramenta é vinculada ao modelo, o modelo pode escolher chamar a ferramenta retornando uma saída estruturada com argumentos da ferramenta. Usamos o método `bind_tools` para aumentar um LLM com ferramentas.

![tool-img](img/tool_call_detail.png)

Provedores frequentemente têm [parâmetros como `tool_choice`](https://python.langchain.com/docs/how_to/tool_choice/) para forçar a chamada de ferramentas específicas. `any` selecionará pelo menos uma das ferramentas.

Além disso, podemos [definir `parallel_tool_calls=False`](https://python.langchain.com/docs/how_to/tool_calling_parallel/) para garantir que o modelo só chamará uma ferramenta por vez (funcionalidade suportada apenas pela OpenAI e Anthropic).

In [ ]:
# Conectar ferramentas a um modelo de chat
model_with_tools = llm.bind_tools([write_email])

# O modelo agora será capaz de chamar ferramentas
output = model_with_tools.invoke("Rascunhe uma resposta para meu chefe (chefe@empresa.com.br) sobre a reunião de amanhã")

In [ ]:
type(output)

In [ ]:
output

In [ ]:
# Extrair chamadas de ferramenta e executá-las
if output.tool_calls:
	args = output.tool_calls[0]['args']
	args
else:
	print("Nenhuma chamada de ferramenta encontrada na resposta do modelo.")
	args = None
	args

In [ ]:
# Extrair os argumentos da chamada de ferramenta do output
if hasattr(output, "tool_calls") and output.tool_calls:
	args = output.tool_calls[0]['args']
	result = write_email.invoke(args)
	Markdown(result)
else:
	print("Nenhuma chamada de ferramenta encontrada na resposta do modelo.")

![basic_prompt](img/tool_call.png)

## Fluxos de Trabalho
 
Existem muitos padrões para construir aplicações com LLMs.

[Podemos incorporar chamadas de LLM em fluxos de trabalho pré-definidos](https://langchain-ai.github.io/langgraph/tutorials/workflows/), dando ao sistema mais autonomia para tomar decisões.

Como exemplo, poderíamos adicionar uma etapa de roteador para determinar se devemos escrever um email ou não.

![workflow_example](img/workflow_example.png)

## Agentes

Podemos aumentar ainda mais a autonomia, permitindo que o LLM direcione dinamicamente seu próprio uso de ferramentas.

[Agentes](https://langchain-ai.github.io/langgraph/tutorials/workflows/#agent) são tipicamente implementados como chamada de ferramenta em um loop, onde a saída de cada chamada de ferramenta é usada para informar a próxima ação.

![agent_example](img/agent_example.png)

Agentes são bem adequados para problemas abertos onde é difícil prever os passos *exatos* necessários com antecedência.

Fluxos de trabalho são frequentemente apropriados quando o fluxo de controle pode ser facilmente definido com antecedência.

![workflow_v_agent](img/workflow_v_agent.png)

## O que é LangGraph?

[LangGraph](https://langchain-ai.github.io/langgraph/concepts/high_level/) fornece infraestrutura de suporte de baixo nível que fica por baixo de *qualquer* fluxo de trabalho ou agente.

Ele não abstrai prompts ou arquitetura, e fornece alguns benefícios:

- **Controle**: Facilita definir e/ou combinar agentes e fluxos de trabalho.
- **Persistência**: Fornece uma maneira de persistir o estado de um grafo, o que permite tanto memória quanto humano-no-loop.
- **Teste, Depuração e Deployment**: Fornece uma rampa fácil para testar, depurar e fazer deploy de aplicações.

### Controle

LangGraph permite definir sua aplicação como um grafo com:

1. *Estado*: Que informações precisamos rastrear ao longo da aplicação?
2. *Nós*: Como queremos atualizar essas informações ao longo da aplicação?
3. *Arestas*: Como queremos conectar esses nós?

Podemos usar a classe [`StateGraph`](https://langchain-ai.github.io/langgraph/concepts/low_level/#graphs) para inicializar um grafo LangGraph com um objeto [`State`](https://langchain-ai.github.io/langgraph/concepts/low_level/#state).

`State` define o esquema para as informações que queremos rastrear ao longo da aplicação.

Isso pode ser qualquer objeto com `getattr()` em python, como um dicionário, dataclass, ou objeto Pydantic:

- TypeDict é mais rápido mas não suporta padrões
- Dataclass é basicamente tão rápido, suporta sintaxe de ponto `state.foo`, e tem padrões.
- Pydantic é mais lento (especialmente com validadores customizados) mas fornece validação de tipo.

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

class StateSchema(TypedDict):
    request: str
    email: str

workflow = StateGraph(StateSchema)

Cada nó é simplesmente uma função python ou código typescript. Isso nos dá controle total sobre a lógica dentro de cada nó.

Eles recebem o estado atual, e retornam um dicionário para atualizar o estado.

Por padrão, [chaves de estado são sobrescritas](https://langchain-ai.github.io/langgraph/how-tos/state-reducers/).

No entanto, você pode [definir lógica de atualização customizada](https://langchain-ai.github.io/langgraph/concepts/low_level/#reducers).

![nodes_edges](img/nodes_edges.png)

In [ ]:
def write_email_node(state: StateSchema) -> StateSchema:
    # Código imperativo que processa a solicitação
    output = model_with_tools.invoke(state["request"])
    args = output.tool_calls[0]['args']
    email = write_email.invoke(args)
    return {"email": email}

Arestas conectam nós.

Especificamos o fluxo de controle adicionando arestas e nós ao nosso grafo de estado.

In [ ]:
workflow = StateGraph(StateSchema)
workflow.add_node("write_email_node", write_email_node)
workflow.add_edge(START, "write_email_node")
workflow.add_edge("write_email_node", END)

app = workflow.compile()

In [ ]:
app.invoke({"request": "Rascunhe uma resposta para meu chefe (chefe@empresa.com.br) sobre a reunião de amanhã"})

O roteamento entre nós pode ser feito [condicionalmente](https://langchain-ai.github.io/langgraph/concepts/low_level/#conditional-edges) usando uma função simples.

O valor de retorno desta função é usado como o nome do nó (ou lista de nós) para o qual enviar o estado a seguir.

Você pode opcionalmente fornecer um dicionário que mapeia a saída `should_continue` para o nome do próximo nó.

In [ ]:
from typing import Literal
from langgraph.graph import MessagesState
from email_assistant.utils import show_graph

def call_llm(state: MessagesState) -> MessagesState:
    """Run LLM"""

    output = model_with_tools.invoke(state["messages"])
    return {"messages": [output]}

def run_tool(state: MessagesState):
    """Performs the tool call"""

    result = []
    for tool_call in state["messages"][-1].tool_calls:
        observation = write_email.invoke(tool_call["args"])
        result.append({"role": "tool", "content": observation, "tool_call_id": tool_call["id"]})
    return {"messages": result}

def should_continue(state: MessagesState) -> Literal["run_tool", "__end__"]:
    """Route to tool handler, or end if Done tool called"""

    # Get the last message
    messages = state["messages"]
    last_message = messages[-1]

    # If the last message is a tool call, check if it's a Done tool call
    if last_message.tool_calls:
        return "run_tool"
    # Otherwise, we stop (reply to the user)
    return END

workflow = StateGraph(MessagesState)
workflow.add_node("call_llm", call_llm)
workflow.add_node("run_tool", run_tool)
workflow.add_edge(START, "call_llm")
workflow.add_conditional_edges("call_llm", should_continue, {"run_tool": "run_tool", END: END})
workflow.add_edge("run_tool", END)

# Run the workflow
app = workflow.compile()

In [ ]:
show_graph(app)

In [ ]:
result = app.invoke({"messages": [{"role": "user", "content": "Draft a response to my boss (boss@company.ai) confirming that I want to attend Interrupt!"}]})
for m in result["messages"]:
    m.pretty_print()

Com esses componentes de baixo nível, você pode construir muitos fluxos de trabalho e agentes diferentes. Veja [este tutorial](https://langchain-ai.github.io/langgraph/tutorials/workflows/)!

Como agentes são um padrão tão comum, o [LangGraph](https://langchain-ai.github.io/langgraph/tutorials/workflows/#pre-built) tem [uma abstração de agente pré-construída](https://langchain-ai.github.io/langgraph/agents/overview/?ref=blog.langchain.dev#what-is-an-agent).

Com o [método pré-construído](https://langchain-ai.github.io/langgraph/tutorials/workflows/#pre-built) do LangGraph, apenas passamos o LLM, ferramentas e prompt.

In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model=llm,
    tools=[write_email],
    prompt="Respond to the user's request using the tools provided."
)

# Run the agent
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Draft a response to my boss (boss@company.ai) confirming that I want to attend Interrupt!"}]}
)

for m in result["messages"]:
    m.pretty_print()

### Persistência

#### Threads

Pode ser muito útil permitir que agentes pausem durante tarefas de longa duração.

LangGraph tem uma camada de persistência integrada, implementada através de checkpointers, para permitir isso.

Quando você compila o grafo com um checkpointer, o checkpointer salva um [checkpoint](https://langchain-ai.github.io/langgraph/concepts/persistence/#checkpoints) do estado do grafo a cada passo.

Checkpoints são salvos em uma thread, que pode ser acessada após a execução do grafo ser completada.

![checkpointer](img/checkpoints.png)

Compilamos o grafo com um [checkpointer](https://langchain-ai.github.io/langgraph/concepts/persistence/#checkpointer-libraries).

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

agent = create_react_agent(
    model=llm,
    tools=[write_email],
    prompt="Responda à solicitação do usuário usando as ferramentas fornecidas.",
    checkpointer=InMemorySaver()
)

config = {"configurable": {"thread_id": "1"}}
result = agent.invoke({"messages": [{"role": "user", "content": "Quais são algumas boas práticas para escrever emails?"}]}, config)

In [ ]:
# Obter o snapshot de estado mais recente
config = {"configurable": {"thread_id": "1"}}
state = agent.get_state(config)
for message in state.values['messages']:
    message.pretty_print()

In [ ]:
# Continue the conversation
result = agent.invoke({"messages": [{"role": "user", "content": "Good, let's use lesson 3 to craft a response to my boss confirming that I want to attend Interrupt"}]}, config)
for m in result['messages']:
    m.pretty_print()

In [ ]:
# Continue the conversation
result = agent.invoke({"messages": [{"role": "user", "content": "I like this, let's write the email to boss@company.ai"}]}, config)
for m in result['messages']:
    m.pretty_print()

#### Interrupts

In LangGraph, we can also use [interrupts](https://langchain-ai.github.io/langgraph/how-tos/human_in_the_loop/wait-user-input/) to stop graph execution at specific points.

Often this is used to collect input from a user and continue execution with collected input.

In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END

from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import InMemorySaver

class State(TypedDict):
    input: str
    user_feedback: str

def step_1(state):
    print("---Step 1---")
    pass

def human_feedback(state):
    print("---human_feedback---")
    feedback = interrupt("Please provide feedback:")
    return {"user_feedback": feedback}

def step_3(state):
    print("---Step 3---")
    pass

builder = StateGraph(State)
builder.add_node("step_1", step_1)
builder.add_node("human_feedback", human_feedback)
builder.add_node("step_3", step_3)
builder.add_edge(START, "step_1")
builder.add_edge("step_1", "human_feedback")
builder.add_edge("human_feedback", "step_3")
builder.add_edge("step_3", END)

# Set up memory
memory = InMemorySaver()

# Add
graph = builder.compile(checkpointer=memory)

In [ ]:
show_graph(graph)

In [ ]:
# Input
initial_input = {"input": "hello world"}

# Thread
thread = {"configurable": {"thread_id": "1"}}

# Run the graph until the first interruption
for event in graph.stream(initial_input, thread, stream_mode="updates"):
    print(event)
    print("\n")

Para retomar de uma interrupção, podemos usar [o objeto `Command`](https://langchain-ai.github.io/langgraph/how-tos/command/). 

Vamos usá-lo para retomar o grafo do estado interrompido, passando o valor para retornar da chamada de interrupção para `resume`.

In [ ]:
# Continue the graph execution
for event in graph.stream(
    Command(resume="go to step 3!"),
    thread,
    stream_mode="updates",
):
    print(event)
    print("\n")

### Tracing

When we are using LangChain or LangGraph, LangSmith logging [will work out of the box](https://docs.smith.langchain.com/observability/how_to_guides/trace_with_langgraph) with the following environment variables set:

```
export LANGSMITH_TRACING=true
export LANGSMITH_API_KEY="<your-langsmith-api-key>"
```

Here is the LangSmith trace from above agent execution:

https://smith.langchain.com/public/6f77014f-d054-44ed-aa2c-8b06ceab689f/r

We can see that the agent is able to continue the conversation from the previous state because we used a checkpointer.

Também podemos implantar nosso grafo usando a [LangGraph Platform](https://langchain-ai.github.io/langgraph/concepts/langgraph_platform/). 

Isso cria um servidor [com uma API](https://langchain-ai.github.io/langgraph/cloud/reference/api/api_ref.html) que podemos usar para interagir com nosso grafo e um IDE interativo, o LangGraph [Studio](https://langchain-ai.github.io/langgraph/concepts/langgraph_studio/).

Precisamos apenas garantir que nosso projeto tenha [uma estrutura](https://langchain-ai.github.io/langgraph/concepts/application_structure/) como esta:

Here we can see a visualization of the graph as well as the graph state in Studio.

![langgraph_studio](img/langgraph_studio.png)

Also, you can see API docs for the local deployment here:

http://127.0.0.1:2024/docs